In [44]:
from PIL import Image
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import PIL
from sklearn.model_selection import train_test_split

import cv2
import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import os
import torch  
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650 SUPER'

In [40]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # carica le immagini RGB e Depth dal DataFrame usando i path
        img_path = self.data.iloc[index]['path_rgb']
        depth_path = self.data.iloc[index]['path_depth']
        rgb_image = PIL.Image.open(img_path).convert('RGB')
        depth_np = np.load(depth_path)
        
        # converte le immagini in tensori PyTorch
        rgb_tensor = transforms.ToTensor()(rgb_image)
        depth_tensor = torch.from_numpy(depth_np).float()
        
        # ritorna una tupla di tensori come input e target per il modello
        return (rgb_tensor, depth_tensor)


In [41]:
csv_path="../Csv/TOY.csv"
df=pd.read_csv(csv_path)
custom_dataset = MyDataset(df)

In [42]:
class ImageConverter(nn.Module):
    def __init__(self):
        super(ImageConverter, self).__init__()
        self.encoder = nn.Sequential( # Encoder
            # una convoluzione 2D con un kernel di dimensione 3x3, stride 1 e padding 1 che riceve in input immagini RGB con 3 canali e produce 32 feature map di output.
            nn.Conv2d(5, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential( # Decoder
            # una convoluzione trasposta 2D con un kernel di dimensione 3x3, stride 2, padding 1 e output padding 1 che riceve in input feature map di dimensioni ridotte prodotte dall'encoder e produce 64 feature map di output.
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(32, 1, kernel_size=1, stride=1)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [45]:
import torch.optim as optim
import torch.nn.functional as F

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def validate(model, device, val_loader):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += F.mse_loss(output, target, reduction='sum').item()
            
    val_loss /= len(val_loader.dataset)
    print('Validation set: Average loss: {:.4f}\n'.format(val_loss))
    return val_loss

# Iperparametri di addestramento
epochs = 10
lr = 0.001
batch_size = 64

# Carica i dati
train_dataset = MyDataset(df[0:20])
val_dataset = MyDataset(df[20:25])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Definisce il modello, l'ottimizzatore e il device di training
model = ImageConverter().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

# Addestramento e validazione
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, val_loader)


RuntimeError: Given groups=1, weight of size [32, 5, 3, 3], expected input[20, 3, 768, 1024] to have 5 channels, but got 3 channels instead

In [20]:
model = ImageConverter()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
dataset =custom_dataset
num_epochs = 5
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [21]:
for epoch in range(num_epochs):
    epoch_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        inputs = batch['input']
        targets = batch['target']
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss/len(dataloader)))


TypeError: list indices must be integers or slices, not str

In [34]:
for batch in dataloader:
    print(batch.img_rgbs) 

AttributeError: 'list' object has no attribute 'img_rgbs'